# Comparing Reddit user engagement across fake, factual, corrective, and regular news

## Motivation

The recent focus on fake news in Western democracies within social sciences has also given rise to the related notion of astroturfing. As defined by Zhang et al. (2013):

> "Online astroturfing refers to coordinated campaigns where messages supporting a specific agenda are distributed via the Internet. These messages employ deception to create the appearance of being generated by an independent entity."

In this perspective, astroturfing can be viewed as a mechanism that enables the initial spread of a fake news item on social media. By initially associating a fake news story with artificial user activity to mimic grassroots activity, the story may gather increased attention from regular users and achieve greater user engagement over the full duration of its lifespan. 

As an example, a recent study on Twitter found that fake news are 70% more likely to be shared than factual news, and that fake news stories reach the first 1,500 users six times faster than factual news stories (Vosoughi et al., 2018).

While the spread of fake news has been studied extensively on Twitter, few studies have been conducted on Reddit. In this study, then, we will analyze whether the effect of a Reddit post's initial user engagement on total user engagement is dependent on the specific type of news story.

# ! Ref??

> Vosoughi, S., Roy, D., & Aral, S. (2018). The spread of true and false news online. Science, 359(6380), 1146-1151.

> Zhang, J., Carpenter, D., & Ko, M. (2013). Online astroturfing: A theoretical perspective.

## Data

To construct the dataset, we collect 695,147,095 Reddit posts from [Pushshift](https://pushshift.io). This constitutes every submission made between January 2016 and April 2020. We also rely on [the ClaimReview Project](https://www.claimreviewproject.com), a collective of verified journalists and fact-checkers, to obtain structured reviews of news items by scraping 151,025 ClaimReviews from [Google Fact Check Explorer](https://toolbox.google.com/factcheck/explorer) spanning from January 2015 to October 2020.

Merging these two datasets, we obtain two collections of post-review pairs: One representing the Reddit posts that link to articles that have also been reviewed by fact checkers; and another representing the posts that link directly to the corrective review articles themselves.

Since not all reviews include a quantitative truthfulness score, we include only those pairs that are numerically scored as either fake or factual. In total, we obtain 4,772 posts linking to news articles, and 17,058 linking to review articles. Among the news articles, 1,596 claims were deemed true while 3,176 were proven false. Similarly, 4,198 reviews verfied the corresponding claim as true while 12,860 disproved the claim as false.

Note that some news articles may contain multiple claims reviewed separately. In the following, we consider a news story as fake if at least one of the reviews pertaining to it identify a claim as false.

![data](../output/data_diagram_final.png)

## Models

### Post level

### Type level

### Story level

### Subreddit level

# Conclusion